In [1]:
import os
from pydantic import BaseModel, Field
from pydantic_ai import Agent
import nest_asyncio
from neo4j import GraphDatabase
import asyncio
import json
from typing import Literal

nest_asyncio.apply()


In [ ]:
# API Keys für den Agent
os.environ["OPENAI_API_KEY"] = "******"
os.environ["OPENAI_API_KEY"]="**********"
os.environ["AZURE_OPENAI_API_KEY"] = "************"
os.environ["AZURE_OPENAI_ENDPOINT"] = (
    "**********"
)
os.environ["OPENAI_API_VERSION"] = "2025-03-01-preview"
#os.environ["OPENAI_BASE_URL"] = "https://**********de/v1/"

In [3]:
class PoliticianGender(BaseModel):
    """Modell für die Geschlechtsbestimmung von Politikern"""
    full_name: str = Field(description="Der vollständige Name des Politikers")
    geschlecht: Literal["männlich", "weiblich", "unbekannt"] = Field(description="Das Geschlecht: 'männlich', 'weiblich' oder 'unbekannt'")
    confidence: float = Field(description="Vertrauenswert zwischen 0.0 und 1.0")

# Agent für Geschlechtsbestimmung
gender_agent = Agent(
    model="openai:gpt-4o",
    instructions="""
    Du bist ein Experte für deutsche Namen und Geschlechtsbestimmung.
    
    Analysiere den gegebenen Namen und die Beschreibung einer Person und bestimme das Geschlecht.
    
    Antworte nur mit: 'männlich', 'weiblich' oder 'unbekannt'
    """,
    output_type=PoliticianGender
)

print("✅ Pydantic Model und Agent erstellt")


✅ Pydantic Model und Agent erstellt


In [8]:
# Neo4j-Verbindung für das Update der Politiker
NEO4J_URI = "bolt://localhost:7687"
NEO4J_AUTH = ("neo4j", "bundestag_password")

class PoliticianGenderUpdater:
    def __init__(self, uri, auth):
        self.driver = GraphDatabase.driver(uri, auth=auth)
        
    def close(self):
        self.driver.close()
    
    def get_politicians_with_content(self, limit=None):
        """Hole alle Politiker mit ihrem ersten Content-Abschnitt"""
        limit_clause = f"LIMIT {limit}" if limit else ""
        
        query = f"""
        MATCH (p:Politician)
        OPTIONAL MATCH (p)-[:HAS_CONTENT]->(c:Content)
        WITH p, c
        ORDER BY p.full_name, c.id
        WITH p, collect(c)[0] as first_content
        RETURN p.detail_page as detail_page,
               p.full_name as full_name,
               p.firstname as firstname,
               p.lastname as lastname,
               CASE WHEN first_content IS NOT NULL 
                    THEN substring(first_content.section_content, 0, 200)
                    ELSE null 
               END as content_preview,
               p.geschlecht as current_gender
        ORDER BY p.full_name
        {limit_clause}
        """
        
        with self.driver.session() as session:
            result = session.run(query)
            return [{
                "detail_page": record["detail_page"],
                "full_name": record["full_name"],
                "firstname": record["firstname"],
                "lastname": record["lastname"],
                "content_preview": record["content_preview"],
                "current_gender": record["current_gender"]
            } for record in result]
    
    def update_politician_gender(self, detail_page, geschlecht, confidence):
        """Update einen Politiker-Knoten mit Geschlechtsinformationen"""
        with self.driver.session() as session:
            session.run("""
                MATCH (p:Politician {detail_page: $detail_page})
                SET p.geschlecht = $geschlecht,
                    p.geschlecht_confidence = $confidence
            """, detail_page=detail_page, geschlecht=geschlecht, 
                       confidence=confidence)
    
    def get_gender_statistics(self):
        """Hole Statistiken über die Geschlechtsverteilung"""
        with self.driver.session() as session:
            result = session.run("""
                MATCH (p:Politician)
                RETURN p.geschlecht as geschlecht, count(*) as count
                ORDER BY count DESC
            """)
            return [dict(record) for record in result]

# Initialisiere den Updater
updater = PoliticianGenderUpdater(NEO4J_URI, NEO4J_AUTH)
print("✅ Neo4j Updater initialisiert")


✅ Neo4j Updater initialisiert


In [9]:
# Lade Testdaten - erste 10 Politiker
test_politicians = updater.get_politicians_with_content(limit=50)

print(f"📊 Gefundene Politiker: {len(test_politicians)}")
print("\nErste 5 Politiker:")
for i, pol in enumerate(test_politicians[:5]):
    content_info = f"Content: {pol['content_preview'][:100]}..." if pol['content_preview'] else "Kein Content"
    current_gender = pol['current_gender'] if pol['current_gender'] else "Nicht gesetzt"
    print(f"{i+1}. {pol['full_name']} | Aktuell: {current_gender}")
    print(f"   {content_info}")
    print()


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n        MATCH (p:Politician)\n        OPTIONAL MATCH (p)-[:HAS_CONTENT]->(c:Content)\n        WITH p, c\n        ORDER BY p.full_name, c.id\n        WITH p, collect(c)[0] as first_content\n        RETURN p.detail_page as detail_page,\n               p.full_name as full_name,\n               p.firstname as firstname,\n               p.lastname as lastname,\n               CASE WHEN first_content IS NOT NULL \n                    THEN substring(first_content.section_content, 0, 200)\n                    ELSE null \n               END as content_preview,\n               p.geschlecht as current_gender\n        ORDER BY p.full_name\n        LIMIT 50\n        '


📊 Gefundene Politiker: 50

Erste 5 Politiker:
1. Aaron Valent | Aktuell: männlich
   Content: Er ist Mitglied des Landesvorstands Bayern der Partei Die Linke.[7]
Bei der Bundestagswahl 2025 kand...

2. Achim Großmann | Aktuell: männlich
   Content: Dieser Artikel befasst sich mit dem Politiker Achim Großmann. Zum Basketballspieler siehe Achim Gros...

3. Achim Kessler | Aktuell: männlich
   Content: „Schafft die Einheit!“: die Figurenkonstellation in der „Ästhetik des Widerstands“ von Peter Weiss. ...

4. Achim Köhler | Aktuell: männlich
   Content: Achim Köhler auf abgeordnetenwatch.de Profil auf bundestag.de...

5. Achim Post | Aktuell: männlich
   Content: Achim Post (2018)
Achim Post (* 2. Mai 1959 in Rahden) ist ein deutscher Politiker (SPD). Er ist sei...



In [10]:
# Teste mit einem einzelnen Politiker
async def test_gender_detection(id = 0):
    """Teste die Geschlechtserkennung mit einem Politiker"""
    
    if not test_politicians:
        print("❌ Keine Testdaten gefunden!")
        return False
    
    # Nimm den ersten Politiker
    test_pol = test_politicians[id]
    
    print(f"🧪 Test mit: {test_pol['full_name']}")
    print("-" * 50)
    print(f"Vorname: {test_pol['firstname']}")
    print(f"Nachname: {test_pol['lastname']}")
    print(f"Content-Preview: {test_pol['content_preview'][:100] if test_pol['content_preview'] else 'Nicht verfügbar'}")
    
    try:
        # Erstelle den Prompt
        prompt = f"""
        Name: {test_pol['full_name']}
        Vorname: {test_pol['firstname'] or 'Unbekannt'}
        Nachname: {test_pol['lastname'] or 'Unbekannt'}
        
        Beschreibung: {test_pol['content_preview'] if test_pol['content_preview'] else 'Keine Beschreibung verfügbar'}
        
        Bestimme das Geschlecht dieser Person.
        """
        
        # Agent-Anfrage
        result = await gender_agent.run(prompt)
        
        print("\n🤖 Agent-Antwort:")
        print(f"   Geschlecht: {result.output.geschlecht}")
        print(f"   Confidence: {result.output.confidence:.2f}")
        
        return result.output
        
    except Exception as e:
        print(f"❌ Test fehlgeschlagen: {e}")
        return None

# Führe den Test aus
test_result = await test_gender_detection(41)


🧪 Test mit: Albert Schmidt
--------------------------------------------------
Vorname: Albert
Nachname: Schmidt
Content-Preview: Nach dem Abitur am Willibald-Gymnasium in Eichstätt absolvierte Schmidt von 1969 bis 1976 ein Pädago

🤖 Agent-Antwort:
   Geschlecht: männlich
   Confidence: 0.98


In [11]:
async def process_politicians_gender(politicians_list, batch_size=50):
    """Verarbeite alle Politiker in Batches"""
    
    print(f"🚀 Starte Geschlechts-Erkennung für {len(politicians_list)} Politiker...")
    print(f"📦 Batch-Größe: {batch_size}")
    print("=" * 70)
    
    success_count = 0
    error_count = 0
    skip_count = 0
    results = []
    
    for i in range(0, len(politicians_list), batch_size):
        batch = politicians_list[i:i+batch_size]
        batch_num = (i // batch_size) + 1
        total_batches = (len(politicians_list) + batch_size - 1) // batch_size
        
        print(f"\n📦 Batch {batch_num}/{total_batches} ({len(batch)} Politiker)")
        print("-" * 50)
        
        for pol in batch:
            try:
                # Überspringe bereits verarbeitete Politiker
                if pol['current_gender']:
                    print(f"   ⏭️  {pol['full_name']} (bereits verarbeitet: {pol['current_gender']})")
                    skip_count += 1
                    continue
                
                print(f"   📝 Verarbeite: {pol['full_name']}")
                
                # Erstelle Prompt
                prompt = f"""
                Name: {pol['full_name']}
                Vorname: {pol['firstname'] or 'Unbekannt'}
                Nachname: {pol['lastname'] or 'Unbekannt'}
                
                Beschreibung: {pol['content_preview'] if pol['content_preview'] else 'Keine Beschreibung verfügbar'}
                
                Bestimme das Geschlecht dieser Person.
                """
                
                # Agent-Anfrage
                result = await gender_agent.run(prompt)
                
                # Update in Neo4j
                updater.update_politician_gender(
                    pol['detail_page'],
                    result.output.geschlecht,
                    result.output.confidence
                )
                
                print(f"      ✅ {result.output.geschlecht} (Confidence: {result.output.confidence:.2f})")
                
                results.append({
                    'full_name': pol['full_name'],
                    'geschlecht': result.output.geschlecht,
                    'confidence': result.output.confidence
                })
                
                success_count += 1
                
            except Exception as e:
                print(f"      ❌ Fehler bei {pol['full_name']}: {e}")
                error_count += 1
        
        # Kurze Pause zwischen Batches
        if i + batch_size < len(politicians_list):
            await asyncio.sleep(1)
    
    print("\n" + "=" * 70)
    print("🎉 Verarbeitung abgeschlossen!")
    print(f"   ✅ Erfolgreich: {success_count}")
    print(f"   ⏭️  Übersprungen: {skip_count}")
    print(f"   ❌ Fehler: {error_count}")
    print(f"   📊 Gesamt: {len(politicians_list)}")
    
    return results, success_count, error_count, skip_count


In [12]:
# Teste mit einem kleinen Batch (erste 20 Politiker)
if test_result:
    print("✅ Einzeltest erfolgreich! Starte Test-Batch...")
    
    # Lade mehr Testdaten
    test_batch = updater.get_politicians_with_content(limit=20)
    
    print(f"\n🧪 Test-Batch mit {len(test_batch)} Politikern")
    print("\n💡 Um den Test-Batch auszuführen, entkommentiere die nächste Zeile:")
    
    # Uncomment to run test batch:
    test_results, success, errors, skipped = await process_politicians_gender(test_batch, batch_size=5)
    
else:
    print("❌ Einzeltest fehlgeschlagen. Bitte überprüfe die Konfiguration.")


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n        MATCH (p:Politician)\n        OPTIONAL MATCH (p)-[:HAS_CONTENT]->(c:Content)\n        WITH p, c\n        ORDER BY p.full_name, c.id\n        WITH p, collect(c)[0] as first_content\n        RETURN p.detail_page as detail_page,\n               p.full_name as full_name,\n               p.firstname as firstname,\n               p.lastname as lastname,\n               CASE WHEN first_content IS NOT NULL \n                    THEN substring(first_content.section_content, 0, 200)\n                    ELSE null \n               END as content_preview,\n               p.geschlecht as current_gender\n        ORDER BY p.full_name\n        LIMIT 20\n        '


✅ Einzeltest erfolgreich! Starte Test-Batch...

🧪 Test-Batch mit 20 Politikern

💡 Um den Test-Batch auszuführen, entkommentiere die nächste Zeile:
🚀 Starte Geschlechts-Erkennung für 20 Politiker...
📦 Batch-Größe: 5

📦 Batch 1/4 (5 Politiker)
--------------------------------------------------
   ⏭️  Aaron Valent (bereits verarbeitet: männlich)
   ⏭️  Achim Großmann (bereits verarbeitet: männlich)
   ⏭️  Achim Kessler (bereits verarbeitet: männlich)
   ⏭️  Achim Köhler (bereits verarbeitet: männlich)
   ⏭️  Achim Post (bereits verarbeitet: männlich)

📦 Batch 2/4 (5 Politiker)
--------------------------------------------------
   ⏭️  Adalbert Hudak (bereits verarbeitet: männlich)
   ⏭️  Adam Balten (bereits verarbeitet: männlich)
   ⏭️  Adelheid Tröscher (bereits verarbeitet: weiblich)
   ⏭️  Adolf Bieringer (bereits verarbeitet: männlich)
   ⏭️  Adolf Cillien (bereits verarbeitet: männlich)

📦 Batch 3/4 (5 Politiker)
--------------------------------------------------
   ⏭️  Adolf Feuring

In [13]:
# Vollständige Verarbeitung aller Politiker
async def run_full_processing():
    """Verarbeite alle Politiker in der Datenbank"""
    
    print("🚨 ACHTUNG: Dies wird ALLE Politiker in der Datenbank verarbeiten! 🚨")
    print("⏱️  Dies kann mehrere Stunden dauern...")
    print()
    
    # Lade alle Politiker
    all_politicians = updater.get_politicians_with_content()
    
    print(f"📊 Gefunden: {len(all_politicians)} Politiker")
    
    # Zähle bereits verarbeitete
    already_processed = sum(1 for pol in all_politicians if pol['current_gender'])
    to_process = len(all_politicians) - already_processed
    
    print(f"   ✅ Bereits verarbeitet: {already_processed}")
    print(f"   📝 Zu verarbeiten: {to_process}")
    
    if to_process == 0:
        print("🎉 Alle Politiker sind bereits verarbeitet!")
        return
    
    # Verarbeitung starten
    results, success, errors, skipped = await process_politicians_gender(
        all_politicians, 
        batch_size=50
    )
    
    # Speichere Ergebnisse
    with open('gender_detection_results.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print(f"\n💾 Ergebnisse gespeichert in: gender_detection_results.json")
    
    return results

print("💡 Um die vollständige Verarbeitung zu starten, entkommentiere die nächste Zeile:")
print("⚠️  WARNUNG: Dies kann sehr lange dauern und viele API-Calls verbrauchen!")

# Uncomment to run full processing:
full_results = await run_full_processing()


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n        MATCH (p:Politician)\n        OPTIONAL MATCH (p)-[:HAS_CONTENT]->(c:Content)\n        WITH p, c\n        ORDER BY p.full_name, c.id\n        WITH p, collect(c)[0] as first_content\n        RETURN p.detail_page as detail_page,\n               p.full_name as full_name,\n               p.firstname as firstname,\n               p.lastname as lastname,\n               CASE WHEN first_content IS NOT NULL \n                    THEN substring(first_content.section_content, 0, 200)\n                    ELSE null \n               END as content_preview,\n               p.geschlecht as current_gender\n        ORDER BY p.full_name\n        \n        '


💡 Um die vollständige Verarbeitung zu starten, entkommentiere die nächste Zeile:
⚠️  WARNUNG: Dies kann sehr lange dauern und viele API-Calls verbrauchen!
🚨 ACHTUNG: Dies wird ALLE Politiker in der Datenbank verarbeiten! 🚨
⏱️  Dies kann mehrere Stunden dauern...

📊 Gefunden: 4465 Politiker
   ✅ Bereits verarbeitet: 273
   📝 Zu verarbeiten: 4192
🚀 Starte Geschlechts-Erkennung für 4465 Politiker...
📦 Batch-Größe: 50

📦 Batch 1/90 (50 Politiker)
--------------------------------------------------
   ⏭️  Aaron Valent (bereits verarbeitet: männlich)
   ⏭️  Achim Großmann (bereits verarbeitet: männlich)
   ⏭️  Achim Kessler (bereits verarbeitet: männlich)
   ⏭️  Achim Köhler (bereits verarbeitet: männlich)
   ⏭️  Achim Post (bereits verarbeitet: männlich)
   ⏭️  Adalbert Hudak (bereits verarbeitet: männlich)
   ⏭️  Adam Balten (bereits verarbeitet: männlich)
   ⏭️  Adelheid Tröscher (bereits verarbeitet: weiblich)
   ⏭️  Adolf Bieringer (bereits verarbeitet: männlich)
   ⏭️  Adolf Cillien (be

In [ ]:
# Zeige aktuelle Statistiken
def show_gender_statistics():
    """Zeige die aktuelle Geschlechtsverteilung"""
    
    stats = updater.get_gender_statistics()
    
    print("📊 Aktuelle Geschlechtsverteilung:")
    print("=" * 40)
    
    total = sum(stat['count'] for stat in stats)
    
    for stat in stats:
        gender = stat['geschlecht'] if stat['geschlecht'] else 'Nicht gesetzt'
        count = stat['count']
        percentage = (count / total) * 100 if total > 0 else 0
        
        print(f"   {gender:15}: {count:4d} ({percentage:5.1f}%)")
    
    print(f"\n   {'Gesamt':15}: {total:4d} (100.0%)")
    
    return stats

current_stats = show_gender_statistics()


In [ ]:
# Überprüfung: Zeige einige aktualisierte Politiker
def show_sample_results(limit=10):
    """Zeige eine Stichprobe der aktualisierten Politiker"""
    
    with updater.driver.session() as session:
        result = session.run("""
            MATCH (p:Politician)
            WHERE p.geschlecht IS NOT NULL
            RETURN p.full_name as name,
                   p.geschlecht as geschlecht,
                   p.geschlecht_confidence as confidence,
            ORDER BY p.geschlecht_confidence DESC
            LIMIT $limit
        """, limit=limit)
        
        samples = [dict(record) for record in result]
    
    print(f"📋 Stichprobe der {len(samples)} besten Ergebnisse:")
    print("=" * 60)
    
    for i, sample in enumerate(samples, 1):
        conf = sample['confidence'] if sample['confidence'] else 0
        print(f"{i:2d}. {sample['name']}")
        print(f"    Geschlecht: {sample['geschlecht']} (Confidence: {conf:.2f})")
        print()
    
    return samples

sample_results = show_sample_results()


In [ ]:
# Cleanup: Schließe die Datenbankverbindung
updater.close()
print("🔒 Datenbankverbindung geschlossen.")
print("\n✅ Gender-Enrichment Notebook abgeschlossen!")
